In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsOneClassifier as OVO
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from classification_utilities import display_cm, display_adj_cm
%matplotlib inline

# Notebook Musings - ophiolite/Stef Bernosky

Access data...

In [211]:
train = pd.read_csv('training_data.csv')
test = pd.read_csv('validation_data_nofacies.csv')
no_facies = pd.read_csv('nofacies_data.csv')

Look at what the data structure looks for in the test data w/o facies

In [212]:
no_facies.head()

,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
0,A1 SH,STUART,2808.0,66.276,0.630,3.3,10.65,3.591,1,1.000
1,A1 SH,STUART,2808.5,77.252,0.585,6.5,11.95,3.341,1,0.978
2,A1 SH,STUART,2809.0,82.899,0.566,9.4,13.60,3.064,1,0.956
3,A1 SH,STUART,2809.5,80.671,0.593,9.5,13.25,2.977,1,0.933
4,A1 SH,STUART,2810.0,75.971,0.638,8.7,12.35,3.020,1,0.911


Well Name should be dropped, as should depth. Formation name can hold a lot of information. Can be misleading, but generally has a lot of predictive power. Create dummy variables to convert Formations to integers.

In [213]:
facies_labels = ['SS', 'CSiS', 'FSiS', 'SiSh', 'MS',
                 'WS', 'D','PS', 'BS']
def coding(col, codeDict):
    colCoded = pd.Series(col, copy=True)
    for key, value in codeDict.items():
        colCoded.replace(key, value, inplace=True)
    return colCoded

Let's see what formations exist first.

In [230]:
pd.value_counts(no_facies['Formation'])

A1 LM    178
C LM     101
B5 LM     99
B1 LM     75
B2 LM     58
C SH      57
A1 SH     43
B3 SH     42
B4 SH     39
B2 SH     34
B1 SH     32
B4 LM     30
B3 LM     25
B5 SH     17
Name: Formation, dtype: int64

In [214]:
pd.value_counts(train["Formation"])

C LM     483
A1 LM    441
A1 SH    336
C SH     305
B5 LM    286
B1 SH    277
B3 SH    220
B4 SH    217
B2 SH    175
B1 LM    152
B2 LM    133
B5 SH     79
B4 LM     71
B3 LM     57
Name: Formation, dtype: int64

Now code the formations for feature engineering...

In [215]:
train['Formations'] = coding(train["Formation"], {'C LM':1,'A1 LM':2, 'A1 SH':3, 'C SH':4, 'B5 LM':5
                                                 , 'B1 SH':6, 'B3 SH':7, 'B4 SH':8, 'B2 SH':9
                                                 , 'B1 LM':10, 'B2 LM':11, 'B5 SH':12, 'B4 LM':13
                                                 , 'B3 LM':14})

Set target and features for modeling. Split into training and test data splits.

In [216]:
y = train.pop('Facies')
train = train.drop(['Well Name', 'Formation', 'Depth'], axis=1)
X = train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Look at the new model features dataframe...

In [218]:
train.head()

,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,Formations
0,77.45,0.664,9.9,11.915,4.6,1,1.000,3
1,78.26,0.661,14.2,12.565,4.1,1,0.979,3
2,79.05,0.658,14.8,13.050,3.6,1,0.957,3
3,86.10,0.655,13.9,13.115,3.5,1,0.936,3
4,74.58,0.647,13.5,13.300,3.4,1,0.915,3


First pass model, one vs. one classification in sklearn. 

In [219]:
model = OVO(LinearSVC(random_state=0))
model.fit(X_train,y_train)

OneVsOneClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1)

Second model, Random Forest Classifier out of the box...

In [220]:
model2 = RandomForestClassifier()
model2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [221]:
model2_predict = model2.predict(X_test)

In [222]:
predicted = model.predict(X_test)

Confusion Matrix for OVO model

In [223]:
conf = confusion_matrix(y_test, predicted)
display_cm(conf, facies_labels, hide_zeros=True)

     Pred    SS  CSiS  FSiS  SiSh    MS    WS     D    PS    BS Total
     True
       SS     5    14    32                                        51
     CSiS     2    22   126           1     1                     152
     FSiS           4   113     1                                 118
     SiSh                 1    17          20                      38
       MS                 2     3     3    33           1          42
       WS                 1     4     2    87          11     2   107
        D                 1     3           8     4     1          17
       PS                 2     3     3    45          28     4    85
       BS                       1           6           4    26    37


In [224]:
def accuracy(conf):
    total_correct = 0.
    nb_classes = conf.shape[0]
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
    acc = total_correct/sum(sum(conf))
    return acc

In [225]:
adjacent_facies = np.array([[1], [0,2], [1], [4], [3,5], [4,6,7], [5,7], [5,6,8], [6,7]])

def accuracy_adjacent(conf, adjacent_facies):
    nb_classes = conf.shape[0]
    total_correct = 0.
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
        for j in adjacent_facies[i]:
            total_correct += conf[i][j]
    return total_correct / sum(sum(conf))

In [226]:
print('Facies classification accuracy = %f' % accuracy(conf))
print('Adjacent facies classification accuracy = %f' % accuracy_adjacent(conf, adjacent_facies))

Facies classification accuracy = 0.471406
Adjacent facies classification accuracy = 0.868624


Confusion Matrix for Random Forest Model. Much better performing out of the box for classification.

In [227]:
conf = confusion_matrix(y_test, model2_predict)
display_cm(conf, facies_labels, hide_zeros=True)

     Pred    SS  CSiS  FSiS  SiSh    MS    WS     D    PS    BS Total
     True
       SS    30    20     1                                        51
     CSiS     5   119    26     1           1                     152
     FSiS     1    26    91                                       118
     SiSh                 1    30     1     4           2          38
       MS           1     1     3    23     8           6          42
       WS     1           1    11    10    58     1    24     1   107
        D                 1                 2    12     2          17
       PS           1           2     2    10          70          85
       BS                       1     1     3     1          31    37


In [228]:
print('Facies classification accuracy = %f' % accuracy(conf))
print('Adjacent facies classification accuracy = %f' % accuracy_adjacent(conf, adjacent_facies))

Facies classification accuracy = 0.717156
Adjacent facies classification accuracy = 0.931994


In [229]:
f1_score(y_test, model2_predict, average='weighted')

0.71454294336391944

# High Level thoughts:
* Reproducibility?
Ok when you have formation names, but on a new region of the field may not be very sensitive to facies changes within the formation.
* Feature Engineering - much more can be done here. This is the half an afternoon example so far...
* Parameter Optimization - still needs to be done, however, in my experience the baseline model tends to be very well optimized in Random Forests.
* Model selection - need to check with another couple classification models to see how it holds up
* Cross-validation - Need to see how it holds up with 10-fold cross validation splits to see which model is most stable